In [12]:
import numpy as np # linear algebra
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [13]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [14]:
train_data = pd.read_csv(r"C:\Users\user\Downloads\titanic\train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
test_data = pd.read_csv(r"C:\Users\user\Downloads\titanic\test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
# Instead of 'Survived', use 'Pclass' as the target
y = train_data["Pclass"]  # This is now your target variable




In [17]:
# Features that may influence Pclass
features = ["Sex", "SibSp", "Parch", "Fare", "Embarked", "Age"]


In [19]:
# Recommended fix
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())

train_data["Fare"] = train_data["Fare"].fillna(train_data["Fare"].median())
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())

train_data["Embarked"] = train_data["Embarked"].fillna(train_data["Embarked"].mode()[0])
test_data["Embarked"] = test_data["Embarked"].fillna(test_data["Embarked"].mode()[0])


In [27]:
# Make sure this is your training data (891 rows)
X = pd.get_dummies(train_data[["Sex", "SibSp", "Parch", "Fare", "Embarked", "Age"]])
X_test = pd.get_dummies(test_data[["Sex", "SibSp", "Parch", "Fare", "Embarked", "Age"]])

# Align test columns to train
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)

# Set your target variable from the training set only!
y = train_data["Pclass"] - 1

# Fit cross_val_score on X and y (same number of rows: 891)
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
rf_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')
rf_acc = rf_scores.mean()
print(f"Random Forest CV Accuracy: {rf_acc:.4f}")



Random Forest CV Accuracy: 0.8945


In [28]:
# Target variable adjusted for XGBoost
y = train_data["Pclass"] - 1

# XGBoost training and validation
xgb_model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                          use_label_encoder=False, eval_metric='mlogloss')
xgb_scores = cross_val_score(xgb_model, X, y, cv=5, scoring='accuracy')
xgb_acc = xgb_scores.mean()
print(f"XGBoost CV Accuracy: {xgb_acc:.4f}")



C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:36:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:36:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:36:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:36:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain,

XGBoost CV Accuracy: 0.9439


C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:36:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [30]:
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_scores = cross_val_score(svm_model, X, y, cv=5, scoring='accuracy')
svm_acc = svm_scores.mean()
print(f"SVM CV Accuracy: {svm_acc:.4f}")


SVM CV Accuracy: 0.7340


In [31]:
# 1. Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
rf_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')
rf_acc = rf_scores.mean()
print(f"Random Forest CV Accuracy: {rf_acc:.4f}")

# 2. XGBoost
xgb_model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                          use_label_encoder=False, eval_metric='mlogloss')
xgb_scores = cross_val_score(xgb_model, X, y, cv=5, scoring='accuracy')
xgb_acc = xgb_scores.mean()
print(f"XGBoost CV Accuracy: {xgb_acc:.4f}")

# 3. SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_scores = cross_val_score(svm_model, X, y, cv=5, scoring='accuracy')
svm_acc = svm_scores.mean()
print(f"SVM CV Accuracy: {svm_acc:.4f}")



# ✅ Step 4: Compare all three accuracies and pick the best model
best_model = None

if rf_acc >= max(xgb_acc, svm_acc):
    best_model = rf_model
    print("✅ Best Model: Random Forest")
elif xgb_acc >= svm_acc:
    best_model = xgb_model
    print("✅ Best Model: XGBoost")
else:
    best_model = svm_model
    print("✅ Best Model: SVM")


# ✅ Step 5: Train the best model on the entire training data
best_model.fit(X, y)

# ✅ Step 6: Predict on test data
predictions = best_model.predict(X_test)

# Since we used y = Pclass - 1, we must reverse it back to original Pclass values
predicted_pclass = predictions + 1

# ✅ Step 7: Save predictions to CSV
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Pclass': predicted_pclass})
output.to_csv('submission.csv', index=False)
print("🎉 Submission file 'submission.csv' created successfully!")


Random Forest CV Accuracy: 0.8945


C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:41:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:41:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:41:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:41:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain,

XGBoost CV Accuracy: 0.9439
SVM CV Accuracy: 0.7340
✅ Best Model: XGBoost
🎉 Submission file 'submission.csv' created successfully!


C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:41:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
